# Building a database for crime reports
---
In this guided project, we will put everything together to build a database for storing data related with crimes that occurred in Boston. This dataset is available in the file `boston.csv`.

The goal of this guided project is for to create a database named `crimes_db` with a table – `boston_crimes` – with appropriate datatypes for storing the data from the `boston.csv` file. You will be creating the table inside a schema named `crimes`. You will also create the `readonly` and `readwrite` groups with the appropriate privileges as we have discussed in this course. Finally, you will also need to create one user for each of these groups.

The following diagram illustrates a high level overview of what we want to achieve:

![achieve](https://dq-content.s3.amazonaws.com/250/goal.png)

## Creating the Crime Database
We will now create our database which will be called as `crime_db` as well as a schema called `crimes`.

First and foremost, let's connect to dataquest's PostgreSQL database since our database does not exist yet.

In [1]:
# Importing the library to connect to PostgreSQL
import psycopg2

try:
    # Creating a connection to the server
    conn = psycopg2.connect('dbname = dq user = dq')
    conn.autocommit = True
    cur = conn.cursor()

    # Creating the database
    cur.execute('''
    CREATE DATABASE crime_db;
    ''')

    # Closing the connection
    conn.close()
except: pass

Since we have now created the database, we can connect to it and create our schema called `crimes`.

In [2]:
try:
    # Creating a connection to the server
    conn = psycopg2.connect('dbname = crime_db user = dq')
    conn.autocommit = True
    cur = conn.cursor()

    # Creating the database
    cur.execute('''
    CREATE SCHEMA crimes;
    ''')
except: pass

## Obtaining the Column Names and Sample
We have successfully created our database and schema, now we will start filling it by first importing our data set into. 

Before we do that, let's gather some data about our crime dataset so that we can more easily select the right datatypes to use in our table.

In [ ]:
# Importing the library to read CSV
import csv

# Reading the file in
with open('boston.csv') as f:
    reader = csv.reader(f)
    col_headers = next(reader)
    first_row = next(reader)

Before we create a table for storing the crime data, we need to identify the proper datatypes for the columns. To help us with that, let's create a function — `get_col_set()` — that given the name of a CSV file and a column index (starting at 0) that computes a Python set with all distinct values contained in that column.

In [ ]:
# Defining a function to find unique values
def get_col_set(csv_filename, col_index):
    values = set()
    with open(csv_filename) as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

# Testing the function
for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7


We will now use the function above to analyze the `description` column. That particular column is located in the second index. We will try and find the maximum length of that column in order to determine the `VARCHAR()` of the column in our database.

In [ ]:
# Using the function for description
desc_uni = get_col_set('boston.csv', 2)

# Finding the maximum length of the entries
max_length = 0
for uni in desc_uni:
    max_length = max(max_length, len(uni))
    
# Printing the result
max_length

We will now create a table called `boston_crimes` inside the `crimes` schema of the `crime_db` database. Before doing anything, let's actually see the header and the first row to determine the data type we will need.

In [ ]:
# Printing the header
print(col_headers, '\n')

# Printing the first row
print(first_row)

The results indicates that:
1. `incident_number` and `offense_code` can be transformed into an integer
2. `description` is a string, but we knew that the maximum length is 58. To be save, we will use `VARCHAR(100)`.
3. `date` can be changed into the date data type
4. `day_of_the_week` is categorical, thus we can use enumerate to replace the values.
5. `lat` and`long` are floating points

In [ ]:
try:
    # Creating enumerated data type
    cur.execute('''
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 
    'Thursday', 'Friday', 'Saturday', 'Sunday');
    ''')

    # Creating the table
    cur.execute('''
    CREATE TABLE crimes.boston_crimes (
            incident_number INTEGER PRIMARY KEY,
            offense_code INTEGER,
            description VARCHAR(100),
            date DATE,
            day_of_the_week weekday,
            lat decimal,
            long decimal
            );
    '''
               )
except: pass

We will now copy the data from our CSV file into our database.

In [ ]:
try:
    # Copying the data in
    with open('boston.csv') as f:
        cur.copy_expert('COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;', f)

    # Checking if it is loaded
    cur.execute("SELECT * FROM crimes.boston_crimes")
    print(len(cur.fetchall()))
except: pass

## Setting the Privileges and Permissions
Now it is time to handle users. Our goal is to create the two user groups that we have learned about: `readonly` and `readwrite`. By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the `public` group and on the `public` schema.

In [ ]:
# Revoking all privileges from the public group
cur.execute('REVOKE ALL ON SCHEMA public FROM public;')

# REvoking privileges of public from the database
cur.execute('REVOKE ALL ON DATABASE crime_db FROM public;')

We will now create two groups:
- `readonly` will only have the ability to select
- `readwrite` will have the ability to select, insert, delete, and update

Logically, we will also grant connection to the database as well as the usage of our schema.

In [ ]:
try:
    # Creating the groups
    cur.execute('CREATE GROUP readonly NOLOGIN;')
    cur.execute('CREATE GROUP readwrite NOLOGIN;')

    # Granting connections to the database
    cur.execute('GRANT CONNECT ON DATABASE crime_db TO readonly;')
    cur.execute('GRANT CONNECT ON DATABASE crime_db TO readwrite;')

    # Granting usage of the schema
    cur.execute('GRANT USAGE ON SCHEMA crimes TO readonly;')
    cur.execute('GRANT USAGE ON SCHEMA crimes TO readwrite;')

    # Changing the permissions
    cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
    cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")
except: pass

Let's wrap up the project by creating one user in each group. We will need to create each user and then assign them to each group.

In [ ]:
try:
    # Creating the data_analyst user
    cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
    cur.execute("GRANT readonly TO data_analyst")

    # Creating the data_scientist user
    cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
    cur.execute("GRANT readwrite TO data_scientist")
except: pass

## Testing the Privilleges
It is a good practice to test that everything is configured as expected when you finish setting up the database. 

We can use SQL queries to check whether the objects have been created and that users and groups have the right privileges. This requires you to know the Postgres internal tables. We can query the `pg_roles` table to inspect privileges related to the database and the `information_schema.table_privileges` table to inspect table privileges.

In [ ]:
# close the old connection to test with a brand new connection
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print()
# check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()